# 모듈 임포트 

In [ ]:
import yaml
import ultralytics
from ultralytics import YOLO
from ultralytics import settings
import warnings
import zipfile
import os
import numpy as np
from glob import glob

warnings.filterwarnings('ignore')
settings.update({'tensorboard' : True})

# 1. Custom Data에 대한 yaml 파일 만들기

In [ ]:
#base 경로 설정 (yaml 파일도 이 경로에 저장)
base_path = './base'
#train data 경로 설정
train_path = os.path.join(base_path, 'Training')
#validation data 경로 설정 
val_path = os.path.join(base_path, 'Validation')
#test data 경로 설정
test_path = os.path.join(base_path, 'Test')

In [ ]:
# yaml 파일 생성 및 확인
data = { 'train' : train_path,
         'val' : val_path,
         'test' : test_path,
         'names' : ['CL', 'CC', 'SD', 'BK', 'LP', 'JF', 'JD', 'DS' , 'ETC'],
         'nc' : 9 }

with open(base_path + '/data.yaml', 'w') as f:
    yaml.dump(data, f)

with open(base_path + '/data.yaml', 'r') as f:
    yaml_file = yaml.safe_load(f)
    display(yaml_file)

# 2. 모델 학습

In [ ]:
#setting 확인
ultralytics.checks()

In [ ]:
#COCO 데이터셋으로 사전 학습된 yolov8 nano 모델 로드
model = YOLO('yolov8n.pt') 

In [ ]:
#하이퍼 파라미터 설정
epochs = 100
patience = 10
batch = 32
imgsz = 640

In [ ]:
#모델 학습
train_results = model.train(data=base_path + '/data.yaml', epochs=epochs, patience=patience, batch=batch, imgsz=imgsz, pretrained = True)

# 3. Validation Set과 Test Set에 대하여 성능평가

In [ ]:
#validation set에 대한 성능평가
val_metrics = model.val(split = 'val')
print('map:', val_metrics.box.map)
print('map50:', val_metrics.box.map50)
print('map75:', val_metrics.box.map75) 
print('maps:', val_metrics.box.maps)

In [ ]:
#test set에 대한 성능평가
test_metrics = model.val(split = 'test')
print('map:', test_metrics.box.map)
print('map50:', test_metrics.box.map50)
print('map75:', test_metrics.box.map75) 
print('maps:', test_metrics.box.maps)

# 4. Test Set에 대한 객체 탐지 수행

In [ ]:
# 테스트 이미지
# 테스트 이미지에 대한 객체 탐지 결과는 runs/detect/predict 폴더에 들어감
test_image_list = glob(test_path + '/images/*')

In [ ]:
results = model.predict(source = test_path + '/images', save=True)

In [ ]:
for result in results:
        
    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  
    uniq_cnt_dict = dict(zip(uniq, cnt))

    print('\n{class num:counts} =', uniq_cnt_dict,'\n')

    for c in result.boxes.cls:
        print('class num =', int(c), ', class_name =', model.names[int(c)])